In [10]:
import pandas as pd
import polars as pl
import json

### Interactions

In [ ]:
interact_path = "C:/Users/Usuario/Datos/goodreads_interactions.csv"
new_path = "C:/Users/Usuario/Datos/interactions_filtered.csv"

# Lee el archivo CSV
df = pl.read_csv(interact_path)

# Filtra las filas donde is_read == 0
df_filtrado = df.filter(pl.col('is_read') != 0)

# Guarda el resultado en un nuevo archivo CSV
df_filtrado.write_csv(new_path)

print("Filtrado completado. El archivo filtrado se ha guardado como 'archivo_filtrado.csv'.")


Filtrado completado. El archivo filtrado se ha guardado como 'archivo_filtrado.csv'.


In [ ]:
final_path = "C:/Users/Usuario/Datos/interactions_sample.csv"
user_counts = df_filtrado.groupby('user_id').agg(pl.count('is_read').alias('count'))

# Paso 2: Seleccionar una muestra aleatoria de 20,000 user_id únicos
sampled_user_ids = user_counts.sample(n=20000, with_replacement=False).select('user_id')

# Paso 3: Filtrar el DataFrame original usando estos user_id seleccionados
df_sampled = df_filtrado.join(sampled_user_ids, on='user_id', how='inner')

df_sampled.write_csv(final_path)

# Verificar el resultado
print(df_sampled.shape)
print(df_sampled.head())

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10080\4251902593.py:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  user_counts = df_filtrado.groupby('user_id').agg(pl.count('is_read').alias('count'))


(2699934, 5)
shape: (5, 5)
┌─────────┬─────────┬─────────┬────────┬─────────────┐
│ user_id ┆ book_id ┆ is_read ┆ rating ┆ is_reviewed │
│ ---     ┆ ---     ┆ ---     ┆ ---    ┆ ---         │
│ i64     ┆ i64     ┆ i64     ┆ i64    ┆ i64         │
╞═════════╪═════════╪═════════╪════════╪═════════════╡
│ 102     ┆ 670     ┆ 1       ┆ 5      ┆ 0           │
│ 102     ┆ 998     ┆ 1       ┆ 5      ┆ 0           │
│ 102     ┆ 996     ┆ 1       ┆ 5      ┆ 0           │
│ 102     ┆ 12971   ┆ 1       ┆ 5      ┆ 0           │
│ 102     ┆ 43      ┆ 1       ┆ 5      ┆ 0           │
└─────────┴─────────┴─────────┴────────┴─────────────┘


### Books

In [8]:
df_samples = pl.read_csv('../Data/interactions_sample.csv')

# Supongamos que df_filtrado ya está cargado
# df_filtrado = pl.read_csv('ruta/al/archivo_filtrado.csv')

# Obtener los valores únicos de la columna 'book_id'
unique_book_ids = df_samples.select('book_id').unique()

# Convertir a una lista
book_id_list = unique_book_ids.to_series().to_list()
book_id_list = [str(book_id) for book_id in book_id_list]

# Imprimir el número de book_id únicos y una muestra de los primeros 10
print(f"Número de book_id únicos: {len(book_id_list)}")
print(f"Primeros 10 book_id únicos: {book_id_list[:10]}")
# Guardar la lista como un archivo JSON
with open('book_ids.json', 'w') as file:
    json.dump(book_id_list, file)

Número de book_id únicos: 535192
Primeros 10 book_id únicos: ['414769', '539297', '243554', '1107530', '120872', '126797', '1021811', '3704', '91912', '289577']


In [ ]:
!jq --slurpfile ids book_ids.json ' . as $all | ($ids[0] | map({(.): true}) | add) as $id_lookup | $all | select(.book_id | in($id_lookup)) | {book_id, authors, similar_books, publication_year, title} ' ../../goodreads_books.json > ../Data/libros_filtrados.json

##### Convert to csv

In [26]:
with open('../Data/libros_filtrados.json','r',encoding='utf-8') as f:
    data = f.readlines()

In [27]:
with open('new_libros.json','w',encoding='utf-8') as ff:
    ff.write('[\n')
    for i,d in enumerate(data):
        if d == '}\n':
            ff.write('},\n')
        else:
            ff.write(d)
    ff.write('\n]')

In [28]:
df = pd.read_json('../Data/new_libros.json')

In [30]:
df.to_csv('../Data/filtered_books.csv',index=False)

In [ ]:
book_ids = list(df['book_id'].unique())

#### Filter interactions from books
We have had trouble on getting all the books because the database is to heavy. So now we are going to filter interactions with only the books we got from the previous command executing during many hours.

Ara filtrem per obtenir únicament els llibres que tenim al nostre sample de books

In [ ]:
path_filt = "C:/Users/Usuario/Datos/interactions_filtered.csv"
df_filtrado = pl.read_csv(path_filt)

In [ ]:
df_filtrado = df_filtrado.filter(pl.col("book_id").is_in(book_ids))

In [ ]:
df_filtrado.to_csv('interactions_final.csv',index=False)

### Authors

In [ ]:
df_authors = pd.read_json('goodreads_book_authors.json', lines=True)

In [ ]:
df = df.drop(columns='text_reviews_count')
df.to_csv('authors.csv',index=False)
df

,average_rating,author_id,name,ratings_count
0,3.98,604031,Ronald J. Fields,49
1,4.08,626222,Anita Diamant,546796
2,3.92,10333,Barbara Hambly,122118
3,3.68,9212,Jennifer Weiner,888522
4,3.82,149918,Nigel Pennick,1740
...,...,...,...,...
829524,4.36,197551,Patty Furbush,11
829525,4.33,3988103,Jim Schlinkman,6
829526,4.00,13464507,Rich Jolly,18
829527,3.31,7427847,sr@ mwrGn,13


In [11]:
df = pl.read_csv('../Data/authors.csv')

Filter authors by books